# IMPORT

In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd 
import numpy as np
from sklearn import preprocessing,svm
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, mean_absolute_error, mean_squared_error,f1_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, Normalizer
from itertools import product, combinations
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import VotingClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split
from sklearn.feature_selection import mutual_info_classif,SelectFromModel
from sklearn.decomposition import PCA
import itertools
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
import cvxopt
import matplotlib.mlab as mlab
import seaborn
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif,SelectFromModel
from sklearn.svm import LinearSVC
from itertools import combinations
from sklearn.decomposition import PCA
import time
import shap

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# DATA CREATION

In [ ]:
# Labels
combined_phenotype= '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Metadata/phenotypic_data.csv'

labels_mappings= pd.read_csv(combined_phenotype)
labels_mappings

,Subject,Label
0,40013,0
1,40014,0
2,40017,0
3,40018,0
4,40019,0
...,...,...
759,4167,0
760,4168,0
761,4169,1
762,4170,1


In [ ]:
local_binary_flattened= '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Flattened Local Features/local_binary_flattened.csv'
local_weighted_flattened= '/content/drive/MyDrive/SUPPLEMENTARY CODE/Data/Generated Features/Flattened Local Features/local_weighted_flattened.csv'

In [ ]:
df1=pd.read_csv(local_binary_flattened)
df2=pd.read_csv (local_weighted_flattened)

In [ ]:
df= df1.merge(df2, how='inner', on='Subject')

In [ ]:
subs= list(set(labels_mappings['Subject'].tolist()))

In [ ]:
for sub in subs:
    if(int(sub)<2000):
        continue
    r = df.index[df.Subject == sub]
    mask = labels_mappings['Subject'] == sub
    df.at[r, 'Schizophrenic'] = labels_mappings[mask]['Label'].values[0]

In [ ]:
#take care of subject 3127
r = df.index[df.Subject == 3127]
df=df.drop(r)

# MODEL

In [ ]:
df_original= df[(df['Subject'] < 3000)]

In [ ]:
df_original.fillna(0, inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_original.to_csv('merged features.csv')

In [ ]:
X = df_original.drop(['Schizophrenic'], axis=1)
X = X.drop(['Subject'], axis=1)
y = df_original['Schizophrenic']

## Feature Selection

In [ ]:
importances = mutual_info_classif(X,y)
feat_importances = pd.Series(importances, df.columns[0:len(df.columns)-2])

In [ ]:
feat_importances.sort_values(ascending=False, inplace=True) 

In [ ]:
feat_importances = feat_importances.to_frame()
feat_importances['name'] = feat_importances.index
feat = feat_importances.head(50)

In [ ]:
feat.reset_index(drop=True, inplace=True)
feat

,0,name
0,0.192734,142_avg neighbour degree
1,0.161039,117_load centrality
2,0.160579,112_local clustering
3,0.159658,152_betweenness centrality
4,0.159655,85_load centrality
5,0.158532,73_square clustering
6,0.156196,122_number of cliques
7,0.154679,70_degree
8,0.151850,70_degree centrality
9,0.149884,116_harmonic centrality


In [ ]:
biomarkers_dict = {str(key):0.0 for key in list(range(164))}

In [ ]:
for ind, row in feat.iterrows():
    roi = row['name'][0:3]
    if roi[-2]=='_':
        roi = roi[0:1]
    if roi[-1]=='_':
        roi = roi[0:2]
    biomarkers_dict[roi]= biomarkers_dict[roi]+ row[0]

In [ ]:
biomarkers_max = sorted(biomarkers_dict.items(), key=lambda y: y[1], reverse=True)[:10]

In [ ]:
biomarkers_max

[('4', 0.42419045658287424),
 ('72', 0.4030090549376779),
 ('117', 0.30869550002416357),
 ('70', 0.30652885240809113),
 ('112', 0.2956906831739914),
 ('152', 0.2934084818614171),
 ('116', 0.2921933897272986),
 ('85', 0.289382321283999),
 ('129', 0.26679037775859005),
 ('131', 0.26657903351546497)]

In [ ]:
biomarkers = [i[0] for i in biomarkers_max]

In [ ]:
biomarkers

['4', '72', '117', '70', '112', '152', '116', '85', '129', '131']

Given ROIs above are potential biomarkers
